# Практическое задание к уроку "Сверточные нейронные сети"

1. Обучить рукописную сверточную нейронную сеть (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете cifar-10


In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
classes = ['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

In [3]:
# установка параметров нейросети
batch_size = 512
num_classes = 10
epochs = 5
data_augmentation = False
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'тренировочные примеры')
print(X_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


X_train = X_train / 255.0
X_test = X_test / 255.0

X_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [22]:
from tensorflow.keras.losses import categorical_crossentropy

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(16, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
# model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

# компиляция модели
model.compile(loss=categorical_crossentropy,
              optimizer='SGD',
              metrics=["accuracy"])

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 32, 32, 16)        448       
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 2, 2, 128)       

In [23]:
# Обучаем модель
hist = model.fit(X_train, y_train,
                 epochs=2,
                 batch_size=128,
                 validation_data=(X_test, y_test),
                 verbose=1)

test_loss, test_metric = model.evaluate(X_test, y_test)
print(f"Test loss {test_loss:.4f}, accuracy {test_metric:.2f}%")

Epoch 1/2
391/391 [==============================] - 32s 81ms/step - loss: 2.2804 - accuracy: 0.1472 - val_loss: 2.1027 - val_accuracy: 0.2246
Epoch 2/2
313/313 [==============================] - 2s 6ms/step - loss: 1.9575 - accuracy: 0.2953
Test loss 1.9575, accuracy 0.30%


2. Оценить рост точности при увеличении ширины сети (больше фильтров)
3. оценить рост точности при увеличении глубины сети (больше слоев)

In [24]:
import pandas as pd

filters = [2, 3, 4, 4, 4, 4]
layers = [1, 1, 1, 2, 3, 4]
accuracy = [0.39, 0.39, 0.33, 0.33, 0.33, 0.2953]
losses = [1.6886, 1.7335, 1.8504, 1.8717, 1.8732, 1.9575]
results = {'filters': filters, 'layers': layers, 'accuracy': accuracy, 'loss': losses}

df = pd.DataFrame(data=results)

Результаты исследования в таблице

In [25]:
df

,filters,layers,accuracy,loss
0,2,1,0.3900,1.6886
1,3,1,0.3900,1.7335
2,4,1,0.3300,1.8504
3,4,2,0.3300,1.8717
4,4,3,0.3300,1.8732
5,4,4,0.2953,1.9575


В моем исследовании сначала увеличивалось количество фильтров Conv2D-MaxPooling с 2 до 4:  
в результате наблюдалось падение метрики accuracy и увеличение потерь  

Далее увеличивалось количество полносвязных слоев Dense с 1 до 4:  
Так же, как и с количеством фильтров наблюдалось ухудшение метрики accuracy и увеличение потерь